## Load modules and read JSON

In [1]:
# load modules
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('kafka_project').getOrCreate()

22/11/18 09:06:23 WARN Utils: Your hostname, tars resolves to a loopback address: 127.0.1.1; using 192.168.1.66 instead (on interface wlan0)
22/11/18 09:06:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/18 09:06:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
# read json file
df_laureates = spark.read\
    .format('json')\
    .option('inferSchema', 'true')\
    .option('multiLine', 'true')\
    .load('data/laureates_data.json')

df_laureates.printSchema()

df_laureates.show()

root
 |-- laureates: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- born: string (nullable = true)
 |    |    |-- bornCity: string (nullable = true)
 |    |    |-- bornCountry: string (nullable = true)
 |    |    |-- bornCountryCode: string (nullable = true)
 |    |    |-- died: string (nullable = true)
 |    |    |-- diedCity: string (nullable = true)
 |    |    |-- diedCountry: string (nullable = true)
 |    |    |-- diedCountryCode: string (nullable = true)
 |    |    |-- firstname: string (nullable = true)
 |    |    |-- gender: string (nullable = true)
 |    |    |-- id: string (nullable = true)
 |    |    |-- prizes: array (nullable = true)
 |    |    |    |-- element: struct (containsNull = true)
 |    |    |    |    |-- affiliations: array (nullable = true)
 |    |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |    |-- category: string (nullable = true)
 |    |    |    |    |-- motivation: string (nullable = 

## Flatten the data by exploding arrays and taking elements of nested struct

In [3]:
from pyspark.sql import functions as F

# explode the laureates array to get the nobel_laureates records
df_laureates = df_laureates.withColumn('laureates', F.explode('laureates')).select('laureates')

In [4]:
df_laureates.show()

+--------------------+
|           laureates|
+--------------------+
|{1845-03-27, Lenn...|
|{1853-07-18, Arnh...|
|{1865-05-25, Zonn...|
|{1852-12-15, Pari...|
|{1859-05-15, Pari...|
|{1867-11-07, Wars...|
|{1842-11-12, Lang...|
|{1862-06-07, Pres...|
|{1856-12-18, Chee...|
|{1852-12-19, Stre...|
|{1845-08-16, Holl...|
|{1874-04-25, Bolo...|
|{1850-06-06, Fuld...|
|{1837-11-23, Leid...|
|{1864-01-13, Gaff...|
|{1869-11-30, Sten...|
|{1853-09-21, Gron...|
|{1879-10-09, Pfaf...|
|{1862-07-02, Wigt...|
|{1890-03-31, Adel...|
+--------------------+
only showing top 20 rows



In [5]:
df_laureates.printSchema()

root
 |-- laureates: struct (nullable = true)
 |    |-- born: string (nullable = true)
 |    |-- bornCity: string (nullable = true)
 |    |-- bornCountry: string (nullable = true)
 |    |-- bornCountryCode: string (nullable = true)
 |    |-- died: string (nullable = true)
 |    |-- diedCity: string (nullable = true)
 |    |-- diedCountry: string (nullable = true)
 |    |-- diedCountryCode: string (nullable = true)
 |    |-- firstname: string (nullable = true)
 |    |-- gender: string (nullable = true)
 |    |-- id: string (nullable = true)
 |    |-- prizes: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- affiliations: array (nullable = true)
 |    |    |    |    |-- element: string (containsNull = true)
 |    |    |    |-- category: string (nullable = true)
 |    |    |    |-- motivation: string (nullable = true)
 |    |    |    |-- overallMotivation: string (nullable = true)
 |    |    |    |-- share: string (nullable = true)
 |    |   

In [6]:
# get the columns within nested struct
df_laureates = df_laureates\
    .withColumn('id', F.col('laureates').getItem('id').cast('int'))\
    .withColumn('born', F.col('laureates').getItem('born'))\
    .withColumn('bornCity', F.col('laureates').getItem('bornCity'))\
    .withColumn('bornCountry', F.col('laureates').getItem('bornCountry'))\
    .withColumn('bornCountryCode', F.col('laureates').getItem('bornCountryCode'))\
    .withColumn('died', F.col('laureates').getItem('died'))\
    .withColumn('diedCity', F.col('laureates').getItem('diedCity'))\
    .withColumn('diedCountry', F.col('laureates').getItem('diedCountry'))\
    .withColumn('diedCountryCode', F.col('laureates').getItem('diedCountryCode'))\
    .withColumn('firstname', F.col('laureates').getItem('firstname'))\
    .withColumn('surname', F.col('laureates').getItem('surname'))\
    .withColumn('gender', F.col('laureates').getItem('gender'))\
    .withColumn('prizes', F.col('laureates').getItem('prizes'))\
    .select('id', 'firstname', 'surname', 'gender', 'born', 'bornCity', 'bornCountry', 'bornCountryCode', 'died', 'diedCity', 'diedCountry', 'diedCountryCode', 'prizes')

In [7]:
df_laureates.printSchema()

root
 |-- id: integer (nullable = true)
 |-- firstname: string (nullable = true)
 |-- surname: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- born: string (nullable = true)
 |-- bornCity: string (nullable = true)
 |-- bornCountry: string (nullable = true)
 |-- bornCountryCode: string (nullable = true)
 |-- died: string (nullable = true)
 |-- diedCity: string (nullable = true)
 |-- diedCountry: string (nullable = true)
 |-- diedCountryCode: string (nullable = true)
 |-- prizes: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- affiliations: array (nullable = true)
 |    |    |    |-- element: string (containsNull = true)
 |    |    |-- category: string (nullable = true)
 |    |    |-- motivation: string (nullable = true)
 |    |    |-- overallMotivation: string (nullable = true)
 |    |    |-- share: string (nullable = true)
 |    |    |-- year: string (nullable = true)



In [8]:
df = df_laureates.withColumn('prizes', F.explode('prizes'))
df = df\
    .withColumn('prize_affiliation', F.col('prizes').getItem('affiliations'))\
    .withColumn('prize_category', F.col('prizes').getItem('category'))\
    .withColumn('prize_motivation', F.col('prizes').getItem('motivation'))\
    .withColumn('prize_overallMotivation', F.col('prizes').getItem('overallMotivation'))\
    .withColumn('prize_share', F.col('prizes').getItem('share'))\
    .withColumn('prize_year', F.col('prizes').getItem('year'))\
    .select('id', 'firstname', 'surname', 'gender', 'born', 'bornCity', 'bornCountry', 'bornCountryCode', 'died', 'diedCity', 'diedCountry', 'diedCountryCode', 'prize_affiliation', 'prize_category', 'prize_motivation', 'prize_overallMotivation', 'prize_share', 'prize_year')

df = df.withColumn('prize_affiliation', F.explode('prize_affiliation'))
df.show()

+---+----------------+----------------+------+----------+--------------------+--------------------+---------------+----------+------------+--------------------+---------------+--------------------+--------------+--------------------+-----------------------+-----------+----------+
| id|       firstname|         surname|gender|      born|            bornCity|         bornCountry|bornCountryCode|      died|    diedCity|         diedCountry|diedCountryCode|   prize_affiliation|prize_category|    prize_motivation|prize_overallMotivation|prize_share|prize_year|
+---+----------------+----------------+------+----------+--------------------+--------------------+---------------+----------+------------+--------------------+---------------+--------------------+--------------+--------------------+-----------------------+-----------+----------+
|  1|  Wilhelm Conrad|         Röntgen|  male|1845-03-27|Lennep (now Remsc...|Prussia (now Germ...|             DE|1923-02-10|      Munich|             Germa

In [9]:
df.printSchema()
# the dataframe is now flattened

root
 |-- id: integer (nullable = true)
 |-- firstname: string (nullable = true)
 |-- surname: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- born: string (nullable = true)
 |-- bornCity: string (nullable = true)
 |-- bornCountry: string (nullable = true)
 |-- bornCountryCode: string (nullable = true)
 |-- died: string (nullable = true)
 |-- diedCity: string (nullable = true)
 |-- diedCountry: string (nullable = true)
 |-- diedCountryCode: string (nullable = true)
 |-- prize_affiliation: string (nullable = true)
 |-- prize_category: string (nullable = true)
 |-- prize_motivation: string (nullable = true)
 |-- prize_overallMotivation: string (nullable = true)
 |-- prize_share: string (nullable = true)
 |-- prize_year: string (nullable = true)



In [10]:
# df without prize affiliations

df_without_prize_affiliations = df.drop('prize_affiliation')

In [11]:
from pyspark.sql.types import StructType, StructField, StringType

# although affiliation has type string, it is actually a struct
# so we need to specify its schema and use the to_json function to convert into struct

# sample data
df.select('id', 'prize_affiliation').toPandas().iloc[0][1]

affiliation_schema = StructType([
    StructField("name", StringType(), True),
    StructField("city", StringType(), True),
    StructField("country", StringType(), True)
])

df = df.withColumn('prize_affiliation', F.from_json(F.col('prize_affiliation'), affiliation_schema))

In [12]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- firstname: string (nullable = true)
 |-- surname: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- born: string (nullable = true)
 |-- bornCity: string (nullable = true)
 |-- bornCountry: string (nullable = true)
 |-- bornCountryCode: string (nullable = true)
 |-- died: string (nullable = true)
 |-- diedCity: string (nullable = true)
 |-- diedCountry: string (nullable = true)
 |-- diedCountryCode: string (nullable = true)
 |-- prize_affiliation: struct (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- city: string (nullable = true)
 |    |-- country: string (nullable = true)
 |-- prize_category: string (nullable = true)
 |-- prize_motivation: string (nullable = true)
 |-- prize_overallMotivation: string (nullable = true)
 |-- prize_share: string (nullable = true)
 |-- prize_year: string (nullable = true)



In [13]:
# lets break the prize affiliation struct into its fields
df = df\
    .withColumn('prize_affiliation_name', F.col('prize_affiliation').getItem('name'))\
    .withColumn('prize_affiliation_city', F.col('prize_affiliation').getItem('city'))\
    .withColumn('prize_affiliation_country', F.col('prize_affiliation').getItem('country'))\
    .select('id', 'firstname', 'surname', 'gender', 'born', 'bornCity', 'bornCountry', 'bornCountryCode', 'died', 'diedCity', 'diedCountry', 'diedCountryCode', 'prize_affiliation_name', 'prize_affiliation_city', 'prize_affiliation_country', 'prize_category', 'prize_motivation', 'prize_overallMotivation', 'prize_share', 'prize_year')

df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- firstname: string (nullable = true)
 |-- surname: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- born: string (nullable = true)
 |-- bornCity: string (nullable = true)
 |-- bornCountry: string (nullable = true)
 |-- bornCountryCode: string (nullable = true)
 |-- died: string (nullable = true)
 |-- diedCity: string (nullable = true)
 |-- diedCountry: string (nullable = true)
 |-- diedCountryCode: string (nullable = true)
 |-- prize_affiliation_name: string (nullable = true)
 |-- prize_affiliation_city: string (nullable = true)
 |-- prize_affiliation_country: string (nullable = true)
 |-- prize_category: string (nullable = true)
 |-- prize_motivation: string (nullable = true)
 |-- prize_overallMotivation: string (nullable = true)
 |-- prize_share: string (nullable = true)
 |-- prize_year: string (nullable = true)



In [14]:
# cleaning data
# number of nulls in each column

df.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df.columns]).show()

+---+---------+-------+------+----+--------+-----------+---------------+----+--------+-----------+---------------+----------------------+----------------------+-------------------------+--------------+----------------+-----------------------+-----------+----------+
| id|firstname|surname|gender|born|bornCity|bornCountry|bornCountryCode|died|diedCity|diedCountry|diedCountryCode|prize_affiliation_name|prize_affiliation_city|prize_affiliation_country|prize_category|prize_motivation|prize_overallMotivation|prize_share|prize_year|
+---+---------+-------+------+----+--------+-----------+---------------+----+--------+-----------+---------------+----------------------+----------------------+-------------------------+--------------+----------------+-----------------------+-----------+----------+
|  0|        1|     33|     0|   1|      39|         33|             33|   0|     400|        395|            395|                   262|                   279|                      265|             0| 

In [15]:
df.select('id','firstname', 'prize_affiliation_name', 'prize_category', 'prize_motivation', 'prize_year').where(F.col('prize_affiliation_name').isNull()).show()

df.count() # 1067
df.distinct().count() # 1067
df.dropDuplicates().count() # 1067
# no duplicate rows

+---+--------------------+----------------------+--------------+--------------------+----------+
| id|           firstname|prize_affiliation_name|prize_category|    prize_motivation|prize_year|
+---+--------------------+----------------------+--------------+--------------------+----------+
|  6|               Marie|                  null|       physics|"in recognition o...|      1903|
|159|       Martinus J.G.|                  null|       physics|"for elucidating ...|      1999|
|278|             Kary B.|                  null|     chemistry|"for his inventio...|      1993|
|462|               Henry|                  null|         peace|"for his humanita...|      1901|
|463|            Frédéric|                  null|         peace|"for his lifelong...|      1901|
|464|                Élie|                  null|         peace|"for his untiring...|      1902|
|465|              Albert|                  null|         peace|"for his eminentl...|      1902|
|466|              Randal|    

1067

In [16]:
df_without_prize_affiliations.count() # 1067

1067

All rows are unique in our dataframe. Previously, we performed explode() two times, one for prizes and another for affiliations. The prizes array needed to be exploded, but exploding the affiliations column may cause problems. However, the dataframe without prize affiliations also has 1067 rows, it means that each row of prize_affiliation column had only one element in its array.

Further, null values exist in columns like death, overallMotivation, which makes sense. So there is no need to drop nulls and further operations can be carried out on this data.

## Questions

### Qn. 1. Find the list of countries which have won the nobel prize and the count of nobel prizes in each category (based on prize affiliation country)

In [17]:
laureates_df = df

laureates_df = laureates_df\
    .groupBy('prize_affiliation_country')\
    .pivot('prize_category')\
    .count()

laureates_df.show()
# null means that the rows the countries don't have a prize in that category, so we replace the nulls by 0

laureates_df = laureates_df\
    .fillna(value="No Affiliated Country", subset=['prize_affiliation_country'])\
    .fillna(value=0)

laureates_df = laureates_df\
    .withColumnRenamed('prize_affiliation_country', 'country')
laureates_df.show(5)

country_prize_category_count_df = laureates_df

+-------------------------+---------+---------+----------+--------+-----+-------+
|prize_affiliation_country|chemistry|economics|literature|medicine|peace|physics|
+-------------------------+---------+---------+----------+--------+-----+-------+
|                   Russia|     null|     null|      null|       1| null|      2|
|                   Sweden|        5|        1|      null|       7| null|      4|
|                  Germany|       41|        1|      null|      18| null|     23|
|                   France|       11|        2|      null|      10|    1|     19|
|                     null|        2|        2|       119|       1|  136|      5|
|                Argentina|        1|     null|      null|       1| null|   null|
|                  Belgium|        1|     null|      null|       4| null|      1|
|                  Finland|        1|     null|      null|    null| null|   null|
|                    India|     null|     null|      null|    null| null|      1|
|               

### Qn. 2. Calculate the age at which each laureate received nobel prize, then find the overall average age at the time of receiving nobel prize, number of prizes/percentage by gender and then average age at the time of receiving nobel prize by gender for each country

In [18]:
laureates_df = df

# there is 1 row that does not have born record
laureates_df.where(F.col('born').isNull()).show() # this record is for an organization

# further we need to remove all records that belong to an organization
laureates_df = laureates_df.where(F.col('gender') != 'org')

laureates_df.select('gender').groupBy('gender').count().show(5) #now all organization records are gone

+---+--------------------+-------+------+----+--------+-----------+---------------+----------+--------+-----------+---------------+----------------------+----------------------+-------------------------+--------------+--------------------+-----------------------+-----------+----------+
| id|           firstname|surname|gender|born|bornCity|bornCountry|bornCountryCode|      died|diedCity|diedCountry|diedCountryCode|prize_affiliation_name|prize_affiliation_city|prize_affiliation_country|prize_category|    prize_motivation|prize_overallMotivation|prize_share|prize_year|
+---+--------------------+-------+------+----+--------+-----------+---------------+----------+--------+-----------+---------------+----------------------+----------------------+-------------------------+--------------+--------------------+-----------------------+-----------+----------+
|925|National Dialogue...|   null|   org|null|    null|       null|           null|0000-00-00|    null|       null|           null|        

In [19]:
from pyspark.sql.types import IntegerType

laureates2_df = laureates_df

# udf to calculate age at the time of reception of each person
def calc_age(birth_year, nobel_prize_reception_year):
    """calculate the age of nobel_prize_reception for a nobel laureate"""
    return (nobel_prize_reception_year - birth_year)

age_udf = F.udf(lambda x, y: calc_age(x, y), IntegerType())
laureates2_df = laureates_df\
    .withColumn('born_year', F.substring('born', 0, 4).cast('int'))\
    .withColumn('prize_year', F.col('prize_year').cast('int'))

laureates2_df = laureates2_df\
    .withColumn('age_at_prize_reception', age_udf(F.col('born_year'), F.col('prize_year')))

laureates2_df.select('id', 'born', 'born_year', 'prize_year', 'age_at_prize_reception').show(5)

+---+----------+---------+----------+----------------------+
| id|      born|born_year|prize_year|age_at_prize_reception|
+---+----------+---------+----------+----------------------+
|  1|1845-03-27|     1845|      1901|                    56|
|  2|1853-07-18|     1853|      1902|                    49|
|  3|1865-05-25|     1865|      1902|                    37|
|  4|1852-12-15|     1852|      1903|                    51|
|  5|1859-05-15|     1859|      1903|                    44|
+---+----------+---------+----------+----------------------+
only showing top 5 rows



In [20]:
# after creating age column in table, we use pivoting

laureates3_df = laureates2_df

laureates3_df = laureates3_df\
    .groupBy(F.col('prize_affiliation_country')\
    .alias('country'))\
    .pivot('gender')\
    .agg(F.count('*').alias('count'), F.avg('age_at_prize_reception').alias('avg_age'))

laureates_country_avg_age_df = laureates2_df\
    .groupBy(F.col('prize_affiliation_country')\
    .alias('country'))\
    .agg(F.avg('age_at_prize_reception').alias('country_avg_age'))

# one row in country is null which should be replaced by 'No Affiliated Country'
laureates3_df = laureates3_df.fillna(value='No Affiliated Country', subset=['country'])
laureates_country_avg_age_df = laureates_country_avg_age_df.fillna(value='No Affiliated Country', subset=['country'])

laureates3_df = laureates3_df.join(laureates_country_avg_age_df, 'country', 'inner')
laureates3_df.show(5)

+--------------------+------------+------------------+----------+-----------------+------------------+
|             country|female_count|    female_avg_age|male_count|     male_avg_age|   country_avg_age|
+--------------------+------------+------------------+----------+-----------------+------------------+
|              Russia|        null|              null|         3|70.66666666666667| 70.66666666666667|
|              Sweden|        null|              null|        17|56.11764705882353| 56.11764705882353|
|             Germany|           2|              52.5|        81|54.75308641975309| 54.69879518072289|
|              France|           3|47.666666666666664|        40|            61.75| 60.76744186046512|
|No Affiliated Cou...|          36| 56.94444444444444|       199|64.52261306532664|63.361702127659576|
+--------------------+------------+------------------+----------+-----------------+------------------+
only showing top 5 rows



In [21]:
# lets replace the null values by 0
laureates4_df = laureates3_df.fillna(0)

# round the values of avg age to 2 significant digits
laureates4_df = laureates4_df\
    .withColumn('female_avg_age', F.round(F.col('female_avg_age'), 1))\
    .withColumn('male_avg_age', F.round(F.col('male_avg_age'), 1))\
    .withColumn('country_avg_age', F.round(F.col('country_avg_age'), 1))

# calculate the male percentage and female percentage also
laureates4_df = laureates4_df\
    .withColumn('female_pct', F.round(F.col('female_count')/(F.col('male_count') + F.col('female_count')) * 100, 2))\
    .withColumn('male_pct', F.round(F.col('male_count')/(F.col('male_count') + F.col('female_count')) * 100, 2))

laureates4_df = laureates4_df.select('country', 'female_count', 'male_count', 'female_pct', 'male_pct', 'female_avg_age', 'male_avg_age', 'country_avg_age')
laureates4_df.show(5)

country_prize_gender_count_df = laureates4_df

+--------------------+------------+----------+----------+--------+--------------+------------+---------------+
|             country|female_count|male_count|female_pct|male_pct|female_avg_age|male_avg_age|country_avg_age|
+--------------------+------------+----------+----------+--------+--------------+------------+---------------+
|              Russia|           0|         3|       0.0|   100.0|           0.0|        70.7|           70.7|
|              Sweden|           0|        17|       0.0|   100.0|           0.0|        56.1|           56.1|
|             Germany|           2|        81|      2.41|   97.59|          52.5|        54.8|           54.7|
|              France|           3|        40|      6.98|   93.02|          47.7|        61.8|           60.8|
|No Affiliated Cou...|          36|       199|     15.32|   84.68|          56.9|        64.5|           63.4|
+--------------------+------------+----------+----------+--------+--------------+------------+---------------+
o

### Qn. 3. List of laureates who won nobel prize in each category each year. If there are multiple laureates who won the prize, make an array.

In [22]:
laureates_df = df

# cast column prize_year as int
laureates_df = laureates_df\
    .withColumn('prize_year', F.col('prize_year').cast('int'))

# create laureates_df with concatenated columns
laureates_df = laureates_df\
    .withColumn('fullname', F.concat(F.col('firstname'),F.lit(' '), F.col('surname')))

# use collect list function to collect the list of names
laureates_df = laureates_df\
    .groupBy('prize_year')\
    .pivot('prize_category')\
    .agg(F.collect_list(F.col('fullname')))\
    .orderBy('prize_year')

laureates_df.show(5)

year_category_laureates_df = laureates_df

+----------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+
|prize_year|           chemistry|economics|          literature|            medicine|               peace|             physics|
+----------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+
|      1901|[Jacobus H. van '...|       []|   [Sully Prudhomme]|  [Emil von Behring]|[Henry Dunant, Fr...|[Wilhelm Conrad R...|
|      1902|      [Emil Fischer]|       []|   [Theodor Mommsen]|       [Ronald Ross]|[Élie Ducommun, A...|[Hendrik A. Loren...|
|      1903|  [Svante Arrhenius]|       []|[Bjørnstjerne Bjø...|[Niels Ryberg Fin...|     [Randal Cremer]|[Henri Becquerel,...|
|      1904|[Sir William Ramsay]|       []|[Frédéric Mistral...|       [Ivan Pavlov]|                  []|     [Lord Rayleigh]|
|      1905|  [Adolf von Baeyer]|       []|[Henryk Sienkiewicz]|       [Robert Koch]|[Bertha von Suttner

## Export data to json

In [23]:
country_prize_category_count_df.show(5)

+--------------------+---------+---------+----------+--------+-----+-------+
|             country|chemistry|economics|literature|medicine|peace|physics|
+--------------------+---------+---------+----------+--------+-----+-------+
|              Russia|        0|        0|         0|       1|    0|      2|
|              Sweden|        5|        1|         0|       7|    0|      4|
|             Germany|       41|        1|         0|      18|    0|     23|
|              France|       11|        2|         0|      10|    1|     19|
|No Affiliated Cou...|        2|        2|       119|       1|  136|      5|
+--------------------+---------+---------+----------+--------+-----+-------+
only showing top 5 rows



In [24]:
country_prize_gender_count_df.show(5)

+--------------------+------------+----------+----------+--------+--------------+------------+---------------+
|             country|female_count|male_count|female_pct|male_pct|female_avg_age|male_avg_age|country_avg_age|
+--------------------+------------+----------+----------+--------+--------------+------------+---------------+
|              Russia|           0|         3|       0.0|   100.0|           0.0|        70.7|           70.7|
|              Sweden|           0|        17|       0.0|   100.0|           0.0|        56.1|           56.1|
|             Germany|           2|        81|      2.41|   97.59|          52.5|        54.8|           54.7|
|              France|           3|        40|      6.98|   93.02|          47.7|        61.8|           60.8|
|No Affiliated Cou...|          36|       199|     15.32|   84.68|          56.9|        64.5|           63.4|
+--------------------+------------+----------+----------+--------+--------------+------------+---------------+
o

In [25]:
year_category_laureates_df.show(5)

+----------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+
|prize_year|           chemistry|economics|          literature|            medicine|               peace|             physics|
+----------+--------------------+---------+--------------------+--------------------+--------------------+--------------------+
|      1901|[Jacobus H. van '...|       []|   [Sully Prudhomme]|  [Emil von Behring]|[Henry Dunant, Fr...|[Wilhelm Conrad R...|
|      1902|      [Emil Fischer]|       []|   [Theodor Mommsen]|       [Ronald Ross]|[Élie Ducommun, A...|[Hendrik A. Loren...|
|      1903|  [Svante Arrhenius]|       []|[Bjørnstjerne Bjø...|[Niels Ryberg Fin...|     [Randal Cremer]|[Henri Becquerel,...|
|      1904|[Sir William Ramsay]|       []|[Frédéric Mistral...|       [Ivan Pavlov]|                  []|     [Lord Rayleigh]|
|      1905|  [Adolf von Baeyer]|       []|[Henryk Sienkiewicz]|       [Robert Koch]|[Bertha von Suttner

In [26]:
# write to json files
# {‘split’, ‘records’, ‘index’, ‘columns’, ‘values’, ‘table’}.

country_prize_category_count_df.toPandas().to_json('data/country_prize_category.json', orient='records') # orient 'records' is necessary for our data format
country_prize_gender_count_df.toPandas().to_json('data/country_prize_gender.json', orient='records')
year_category_laureates_df.toPandas().to_json('data/year_category_laureates.json', orient='records')